# Mini tutorial de SQL para científicos

Hoy en día no se puede seguir trabajando con ficheros de texto habiendo tantas alternativas, sobretodo a medida que el tamaño de la información crece y crece y se hace inmanejable tratar ficheros de varios cientos de Mb. Es por ello que hoy vamos a ver por encima cómo podemos hacer consultas, modificar y crear nueva información en una base de datos SQL.

# ¿Qué es SQL?

SQL es un acrónimo para **S**tructured **Q**uery **L**anguage. Es un lenguaje que nos permite acceder a bases de datos SQL (bases de datos relacionales). Un RDBMS (**R**elational **D**ata**B**ase **M**anagement **S**ystem) es un sistema que nos permite acceder, crear, editar y gestionar bases de datos relacionales. Existen RDBMS muy populares como MySQL (MariaDB), PostgreSQL o SQLite. En el presente tutorial vamos a trabajar con SQLite por simplicidad y porque viene disponible con CPython.

# ¿Y Pandas?

Pandas dispone de funcionalidad que nos permite leer y escribir información en bases de datos relacionales.

# Vamos a crear una base de datos SQLite

Las bases de datos SQLite son bases de datos que no necesitan un servidor y que se guardan en disco. Para más información [pulsa aquí](https://duckduckgo.com/?q=sqlite). Podéis inspeccionar la base de datos que vayamos a crear, bastante simple, con [SQLite manager, un addon para firefox](https://addons.mozilla.org/en-US/firefox/addon/sqlite-manager/).

Como siempre, primero importamos todo lo necesario

In [6]:
import sqlite3
import datetime as dt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Mi configuración es la siguiente:

In [7]:
%load_ext version_information
%version_information pandas, numpy, matplotlib

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


Software versions
Python 3.4.0 64bit [GCC 4.8.2]
IPython 3.0.0
OS Linux 3.13.0 24 generic x86_64 with LinuxMint 17 qiana
pandas 0.15.2
numpy 1.9.2
matplotlib 1.4.3
Wed Mar 11 23:17:54 2015 CET

Primero necesitamos poder conectar con la base de datos. Esto es de lo poco que diferirá con respecto a otros RDBMSs.

In [11]:
conn = sqlite3.connect('pybodb.sqlite')

# ejemplo con PostgreSQL usando psycopg2
# import psycopg2
# conn = psycopg2.connect(database='ejemplodb', user='kiko') 

# ejemplo con MS ACCESS usando pyodbc (sí, en el trabajo tengo que usar estas cosas)
# import pyodbc
# conn = pyodbc.connect("Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=ejemplodb;") 

# ...

Ahora que ya tenemos una conexión a la base de datos Pandas se puede encargar del trabajo sucio de 'hablar' con la misma y ayudarnos a interactuar directamente con los datos de la forma habitual y potente de Pandas.

Vamos a crear un DataFrame que usaremos como una tabla para insertar en la base de datos Este DataFrame tendrá una columna de fechas, una de medidas de temperatura promedio diaria (inventada), una de precipitación acumulada en 24h (inventada), una columna con el tipo de sensor que midió la temperatura ese día y una última con el sensor que midió la precipitación.

In [12]:
# fechas para cada día del 2014
fechas = pd.date_range(dt.datetime(2014, 1, 1), dt.datetime(2014, 12, 31))
# Temperatura media diaria durante 2014 en algún lugar del hemisferio Norte
tmed = (np.random.randint(-5, 5, size = 365) + 
        20 * np.cos(np.arange(0 - 180, 365 - 180) * 2 * np.pi / 365) 
        + 10)
# Precipitación acumulada en 24h
prec = (20 * np.abs(np.random.randn(365) * 
        np.cos(np.arange(0, 365) * 2 * np.pi / 365)))
# Sensor que midió la temperatura
marcaT = np.random.choice(['marcaT1', 'marcaT2', 'marcaT3'], size = 365)
# Sensor midió la precipitación
marcaP = np.random.choice(['marcaP1', 'marcaP2'], size = 365)

# Creamos el dataframe y lo guardamos en una tabla llamada 'datos'
df = pd.DataFrame(
        np.array([fechas.values, tmed, prec, marcaT, marcaP]).T,
        columns = ['fecha', 'tmedia','precipitacion','sensorT','sensorP'])
df['fecha'] = pd.to_datetime(df['fecha'])
df.to_sql('datos', con = conn, dtype = {'time': 'TIMESTAMP'})

Vamos a crear una segunda tabla para añadir un poco de complejidad a las consultas que hagamos posteriormente a la base de datos. Esta tabla contendrá información de los sensores usados para las medidas.

In [13]:
# fechas para cada día del 2014
sensores = ['marcaT1', 'marcaT2', 'marcaT3',
            'marcaP1', 'marcaP2']

# Precisión de los sensores
precision = [0.1, 0.5, 1,
             2, 5]

df = pd.DataFrame({'sensores': sensores, 'precision': precision})
df.to_sql('sensores', con = conn)

# Consultando la base de datos

Dentro de SQL tenemos comandos que se pueden integrar en diferentes categorías. Ahora vamos a usar *dql (data query language)* que se usa para hacer consultas a la base de datos. Ven en [esta chuleta para conocer más](http://www.sql-tutorial.net/SQL-Cheat-Sheet.pdf).

Como SQL no hace distinción de mayúsculas y minúsculas usaré mayúsculas para las palabras clave de SQL. Para hacer una petición se usa `SELECT`. Veamos como es una consulta para conocer las tablas que existen en la base de datos:

In [18]:
# Esto es específico para sqlite
print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))
# Para otras BBDD puedes buscar en internet :-)

       name
0     datos
1  sensores


En la consulta anterior hemos usados varias palabras clave: `SELECT`, `FROM`, `WHERE`. Este tipo de consultas serán de los más habitual. Vamos a explotar lo que ya sabemos.

Quiero los datos de todas las columnas de la tabla sensores (y pandas me lo meterá en un DataFrame, maravilloso!!!)

In [20]:
df = pd.read_sql("SELECT * FROM sensores;", conn)

In [22]:
print(df)

   index  precision sensores
0      0        0.1  marcaT1
1      1        0.5  marcaT2
2      2        1.0  marcaT3
3      3        2.0  marcaP1
4      4        5.0  marcaP2


Ahora queremos los datos de precipitación de Junio junto con su fecha. Fijaos que el valor superior del rango no es inclusivo (`BETWEEN '2014-06-01' AND '2014-07-01'` nos da el dato hasta antes de `fecha = '2014-07-01'`):

In [36]:
df = pd.read_sql("SELECT fecha, precipitacion FROM datos WHERE fecha BETWEEN '2014-06-01' AND '2014-07-01';", conn)

In [35]:
print(df)

                  fecha  precipitacion
0   2014-06-01 00:00:00      18.888751
1   2014-06-02 00:00:00      25.410252
2   2014-06-03 00:00:00       9.398012
3   2014-06-04 00:00:00      32.976860
4   2014-06-05 00:00:00       5.386931
5   2014-06-06 00:00:00      17.932470
6   2014-06-07 00:00:00      11.764828
7   2014-06-08 00:00:00      27.100781
8   2014-06-09 00:00:00      13.992309
9   2014-06-10 00:00:00       8.277962
10  2014-06-11 00:00:00       4.256468
11  2014-06-12 00:00:00      12.155609
12  2014-06-13 00:00:00       1.149901
13  2014-06-14 00:00:00      18.031408
14  2014-06-15 00:00:00       5.197137
15  2014-06-16 00:00:00       9.228805
16  2014-06-17 00:00:00       5.325197
17  2014-06-18 00:00:00       6.793255
18  2014-06-19 00:00:00       9.005771
19  2014-06-20 00:00:00      38.267340
20  2014-06-21 00:00:00      12.681727
21  2014-06-22 00:00:00      24.818292
22  2014-06-23 00:00:00      31.630442
23  2014-06-24 00:00:00      22.891158
24  2014-06-25 00:00:00  

Ahora quiero los datos de temperatura de los sensores con una precisión superior a 0.5 (el sensor *marcaT1* es el único que me da precisión superior a 0.5):

In [39]:
df = pd.read_sql("SELECT fecha, tmedia FROM datos WHERE datos.sensorT = 'marcaT1';", conn)

In [40]:
print(df)

                   fecha     tmedia
0    2014-01-03 00:00:00 -13.940023
1    2014-01-04 00:00:00  -7.910427
2    2014-01-08 00:00:00  -9.733159
3    2014-01-15 00:00:00  -5.198654
4    2014-01-18 00:00:00  -7.883750
5    2014-01-28 00:00:00  -6.476142
6    2014-01-29 00:00:00 -10.306145
7    2014-02-11 00:00:00  -8.649881
8    2014-02-12 00:00:00  -1.413343
9    2014-02-23 00:00:00  -1.545832
10   2014-02-26 00:00:00  -4.687451
11   2014-02-27 00:00:00  -4.394876
12   2014-03-01 00:00:00  -4.800573
13   2014-03-03 00:00:00   2.805345
14   2014-03-04 00:00:00   2.112436
15   2014-03-06 00:00:00  -1.265572
16   2014-03-07 00:00:00   4.049144
17   2014-03-10 00:00:00   0.007051
18   2014-03-13 00:00:00  -0.016396
19   2014-03-15 00:00:00   5.643714
20   2014-03-16 00:00:00   5.976199
21   2014-03-20 00:00:00   7.320033
22   2014-03-21 00:00:00   7.658967
23   2014-03-24 00:00:00   3.681308
24   2014-04-02 00:00:00   6.774446
25   2014-04-08 00:00:00   9.830806
26   2014-04-09 00:00:00  16

In [47]:
# La misma consulta de antes pero sin saber la precisión de cada uno de los sensores a priori
q = """
SELECT 
    fecha, tmedia 
FROM 
    datos 
WHERE 
    datos.sensorT = 
    (SELECT 
        sensores 
    FROM 
        sensores 
    WHERE 
        precision < 0.5);"""

df = pd.read_sql(q, conn)

In [48]:
print(df)

                   fecha     tmedia
0    2014-01-03 00:00:00 -13.940023
1    2014-01-04 00:00:00  -7.910427
2    2014-01-08 00:00:00  -9.733159
3    2014-01-15 00:00:00  -5.198654
4    2014-01-18 00:00:00  -7.883750
5    2014-01-28 00:00:00  -6.476142
6    2014-01-29 00:00:00 -10.306145
7    2014-02-11 00:00:00  -8.649881
8    2014-02-12 00:00:00  -1.413343
9    2014-02-23 00:00:00  -1.545832
10   2014-02-26 00:00:00  -4.687451
11   2014-02-27 00:00:00  -4.394876
12   2014-03-01 00:00:00  -4.800573
13   2014-03-03 00:00:00   2.805345
14   2014-03-04 00:00:00   2.112436
15   2014-03-06 00:00:00  -1.265572
16   2014-03-07 00:00:00   4.049144
17   2014-03-10 00:00:00   0.007051
18   2014-03-13 00:00:00  -0.016396
19   2014-03-15 00:00:00   5.643714
20   2014-03-16 00:00:00   5.976199
21   2014-03-20 00:00:00   7.320033
22   2014-03-21 00:00:00   7.658967
23   2014-03-24 00:00:00   3.681308
24   2014-04-02 00:00:00   6.774446
25   2014-04-08 00:00:00   9.830806
26   2014-04-09 00:00:00  16

Podemos decirle que nos pase solo una serie de valores. Por ejemplo, solo quiero los tres valores más altos de precipitación de diciembre:

In [54]:
q = """
SELECT 
    fecha, precipitacion 
FROM 
    datos 
WHERE 
    fecha > '2014-11-30'
ORDER BY
    precipitacion DESC
LIMIT
    3"""

df = pd.read_sql(q, conn)

In [55]:
print(df)

,fecha,precipitacion
0,2014-12-07 00:00:00,40.643008
1,2014-12-10 00:00:00,37.531830
2,2014-12-20 00:00:00,36.307755


En la consulta anterior le hemos pedido que nos ordenase por los valores de precipitación de forma descendente (es decir, por el final) y le hemos pedido que nos limitase la búsqueda a tres valores, los tres valores más altos.

# ¿Os ha picado el gusanillo?

Todo esto era para ver si os picaba un poco el gusanillo y dejáis (dejamos) de usar tanto fichero de texto/excel/csv y usamos opciones más ricas y potentes que existen por ahí.

# Enlaces

* Un [minitutorial sobre SQL](http://opentechschool.github.io/sql-tutorial/index.html) muy ameno.

* Nuestro (inacabado) [tutorial sobre PyTables](http://pybonacci.org/tag/pytables/).

* Y [¿por qué no usar netCDF?](http://pybonacci.org/2012/12/03/por-que-usar-netcdf/)

* Funcionalidad [SQL en pandas](http://pandas.pydata.org/pandas-docs/stable/io.html?highlight=read_sql#io-sql).

# Y cerramos la conexión (literal).

In [56]:
conn.close() # :-)